<a href="https://colab.research.google.com/github/smillinggirl/cnn-dailymail-rag/blob/main/CNN_DAILYMAIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load CNN/DailyMail dataset (only train split)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
import re

def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^a-zA-Z0-9.,!?\'\s]', '', text)  # Remove special characters (keep some punctuation)
    return text

# Apply cleaning to articles & summaries
dataset = dataset.map(lambda example: {
    "article": clean_text(example["article"]),
    "highlights": clean_text(example["highlights"])
})


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

In [ ]:
def chunk_article(article, chunk_size=200):
    """
    Splits an article into chunks of approximately 'chunk_size' words each.
    """
    words = article.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)

    return chunks

# Apply this function to all articles in train set
train_chunks = []
train_summaries = []

for example in dataset:
    article_chunks = chunk_article(example['article'])
    summary = example['highlights']

    for chunk in article_chunks:
        train_chunks.append(chunk)
        train_summaries.append(summary)

# Check example
print(f"Total Chunks Created: {len(train_chunks)}")
print("Example Chunk:")
print(train_chunks[0])
print("\nCorresponding Summary:")
print(train_summaries[0])


Total Chunks Created: 1131566
Example Chunk:
london, england reuters harry potter star daniel radcliffe gains access to a reported 20 million 41.1 million fortune as he turns 18 on monday, but he insists the money won't cast a spell on him. daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. i don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar, he told an australian interviewer earlier this month. i don't think i'll be particularly extravagant. the things i like buying are things that cost about 10 pounds books and cds and dvds. at 18, radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film hostel part ii, currently six places below his number one movie on the uk box offic

In [ ]:
from sentence_transformers import SentenceTransformer

# You can change model if you want - this is a small & fast one
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & works well


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np

# Example: train_chunks should already be created from the chunking step you did
chunk_embeddings = embedding_model.encode(train_chunks, convert_to_numpy=True)

# Now we have all chunks converted into vectors (embeddings)


KeyboardInterrupt: 

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# Work with a sample (optional)
sample_chunks = train_chunks[:1000]

# Batch embedding
chunk_embeddings = embedding_model.encode(
    sample_chunks,
    convert_to_numpy=True,
    batch_size=64
)



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import faiss

embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(chunk_embeddings)


In [ ]:
chunk_to_summary = {}

chunk_index = 0
for example in dataset:
    article_chunks = chunk_article(example['article'])
    summary = example['highlights']

    for chunk in article_chunks:
        chunk_to_summary[chunk_index] = summary
        chunk_index += 1


In [ ]:
def retrieve_best_chunk_and_summary(query):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)
    distances, indices = index.search(query_embedding.reshape(1, -1), 1)  # Top 1 chunk

    best_chunk = train_chunks[indices[0][0]]
    best_summary = chunk_to_summary[indices[0][0]]

    return best_chunk, best_summary


In [ ]:
query = "What did Daniel Radcliffe say about his fortune?"

best_chunk, best_summary = retrieve_best_chunk_and_summary(query)

print("Best Matching Chunk:\n", best_chunk)
print("\nSummary of Full Article:\n", best_summary)


Best Matching Chunk:
 london, england reuters harry potter star daniel radcliffe gains access to a reported 20 million 41.1 million fortune as he turns 18 on monday, but he insists the money won't cast a spell on him. daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. i don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar, he told an australian interviewer earlier this month. i don't think i'll be particularly extravagant. the things i like buying are things that cost about 10 pounds books and cds and dvds. at 18, radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film hostel part ii, currently six places below his number one movie on the uk box office chart. details of how

Retrieved Chunks:
- london, england reuters harry potter star daniel radcliffe gains access to a reported 20 million 41.1 million fortune as he turns 18 on monday, but he insists the money won't cast a spell on him. daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. i don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar, he told an australian interviewer earlier this month. i don't think i'll be particularly extravagant. the things i like buying are things that cost about 10 pounds books and cds and dvds. at 18, radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film hostel part ii, currently six places below his number one movie on the uk box office chart. details of how h